In [2]:
from transformers import BertTokenizerFast, BertForPreTraining,BertForTokenClassification ,Trainer, TrainingArguments
import os
import torch 
import pandas as pd
import datasets

In [3]:
torch.cuda.is_available()

True

In [116]:
from array import array
import numpy as np
def loadData():
    list_subfolders_with_paths = [f.path for f in os.scandir("./annotation_Facebook/") if f.is_dir()]
    data = {"texts": [],"labels":[]}
    for i in list_subfolders_with_paths:
    #print("new File")
        texts=[]
        labels=[]
        with open(f"{i}/admin.conll","r", encoding="utf-8-sig") as file: 
            for j in  file.readlines():
                if(j.strip() == ""):
                    continue
                sp = j.split(" ")
                if("null" in sp[1].rstrip("\n") or "\xa0" in sp[1].rstrip("\n")):
                    print(i)
                texts.append(sp[0])

                labels.append(sp[1].rstrip("\n"))
                #print(f"Text: {sp[0]}")
                #print(f"Label: {sp[1]}")
            data["texts"].append((texts))
            data["labels"].append(( labels))
    
    
    return data

In [117]:
from datasets import Dataset , ClassLabel ,Features ,Sequence ,Value

In [118]:
traindata = loadData()
texts = traindata["texts"]
tags = traindata["labels"]

In [119]:
from collections import Counter
x = [i[2:] if(len(i) > 1) else i for i in list(sum(tags, []))]
di = dict(Counter(x))

z = {}

for i in di.keys():
    if i == "O":
        continue
    z[i] = round ((di[i]/(len(x)-di["O"]))*100,2)
    

z

{'PERSON_FIRSTNAME': 2.89,
 'PERSON_LASTNAME': 2.51,
 'ORGANIZATION': 5.94,
 'LOCATION': 3.43,
 'MISC_FRIENDS': 27.88,
 'LOCATION_CITY': 4.72,
 'LOCATION_COUNTRY': 0.84,
 'MISC_LIKES': 4.49,
 'MISC_POST': 31.38,
 'MISC_COMMENTS': 15.92}

In [120]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [121]:
list(unique_tags)

['I-LOCATION',
 'B-PERSON_FIRSTNAME',
 'I-MISC_POST',
 'I-MISC_COMMENTS',
 'I-ORGANIZATION',
 'I-LOCATION_CITY',
 'I-PERSON_FIRSTNAME',
 'O',
 'I-LOCATION_COUNTRY',
 'I-MISC_FRIENDS',
 'B-MISC_COMMENTS',
 'B-MISC_FRIENDS',
 'B-ORGANIZATION',
 'B-MISC_LIKES',
 'B-MISC_POST',
 'B-LOCATION_COUNTRY',
 'B-LOCATION_CITY',
 'B-PERSON_LASTNAME',
 'B-LOCATION']

In [122]:
dataset = Dataset.from_dict(loadData(),features=Features( {"texts": Sequence(Value("string")),'labels':Sequence(ClassLabel(names=list(unique_tags),num_classes =len(unique_tags)))}))

In [123]:
dataset = dataset.train_test_split(test_size=0.2)

In [124]:
(dataset["train"].features["labels"].feature)

ClassLabel(num_classes=19, names=['I-LOCATION', 'B-PERSON_FIRSTNAME', 'I-MISC_POST', 'I-MISC_COMMENTS', 'I-ORGANIZATION', 'I-LOCATION_CITY', 'I-PERSON_FIRSTNAME', 'O', 'I-LOCATION_COUNTRY', 'I-MISC_FRIENDS', 'B-MISC_COMMENTS', 'B-MISC_FRIENDS', 'B-ORGANIZATION', 'B-MISC_LIKES', 'B-MISC_POST', 'B-LOCATION_COUNTRY', 'B-LOCATION_CITY', 'B-PERSON_LASTNAME', 'B-LOCATION'], names_file=None, id=None)

In [125]:
dataset["train"][0]

{'texts': ['Njomza',
  'Krasniqi',
  '100',
  'Freunde',
  'Freund',
  '/',
  'in',
  'hinzufügen',
  'Nachricht',
  'senden',
  'Steckbrief',
  'Wohnt',
  'in',
  'Köln',
  'Single',
  'Fotos',
  'Alle',
  'Fotos',
  'ansehen',
  'Freunde',
  'Alle',
  'Freunde',
  'ansehen',
  '100',
  'Freunde',
  'Njomza',
  'Krasniqi',
  '\xa0',
  '1',
  'Privatsphäre',
  '·',
  'Impressum',
  '/',
  'Terms',
  '/',
  'NetzDG',
  '/',
  'UrhDaG',
  '·',
  'Werbung',
  '·',
  'Datenschutzinfo',
  '·',
  'Cookies',
  '·',
  '·',
  'Meta',
  '©',
  '2022',
  'Arlind',
  'Krasniqi',
  'Salwan',
  'Alqaidi',
  'Agnesa',
  'Sekiraqa',
  'Daffina',
  'Iimeri',
  'Florian',
  'Florian',
  'Beqiri',
  'Festim',
  'Krasniqi',
  'Kaonna',
  'Krasniqi',
  'Taulanti',
  'Grajqevci',
  'Arian',
  'G',
  'Hoti',
  'Mehr',
  'Beiträge',
  'Filter',
  'Njomza',
  'Krasniqi',
  'hat',
  'ihr',
  'Titelbild',
  'aktualisiert',
  '.',
  '4',
  '.',
  'April',
  '2013',
  '·',
  'Teilen',
  '\xa0',
  '1'],
 'labels': 

In [126]:
label_names = dataset["train"].features[f"labels"].feature.names

In [127]:
len(label_names)

19

In [128]:
#Load Model and Tokenizer
from transformers import AutoTokenizer

tokenizer = BertTokenizerFast.from_pretrained('deepset/gbert-large')
model = BertForTokenClassification.from_pretrained('deepset/gbert-large', num_labels=len(label_names) )

loading file https://huggingface.co/deepset/gbert-large/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/ace1801aadb4eaa6eddd4240e19842e80bf7ed5859519382b3095d9e9de08ff1.4d65bbd3b91f2762e9d2c779d48ab14052439d3fcc8c3d2fe78c7322a9ac8d64
loading file https://huggingface.co/deepset/gbert-large/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/deepset/gbert-large/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/deepset/gbert-large/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/deepset/gbert-large/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/52a68b9ec5a157f2334ff64e6e5d1d49db0f49c1a16a21e74f497d67599ca4d8.bd85f6a0d9c2848260fdda7056a557a7fa213a57b5ed27b530abd20572c0cbf4
loading configuration file https://huggingface.co/deepset/gbert-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformer

In [143]:
def tokenize_and_align_labels(examples):
    #print(examples.keys())
    tokenized_inputs = tokenizer(examples["texts"],max_length= 1024, truncation=True, is_split_into_words=True) # set max length 1024

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [144]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [145]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['texts', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 24
    })
    test: Dataset({
        features: ['texts', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6
    })
})

In [146]:
def split(examples):
    dic ={"input_ids":[],"token_type_ids":[],"attention_mask":[],"labels":[]}
    for j in examples:
        if j == "texts":
            continue
        for i in range(len(examples["input_ids"])):
            arr = np.array_split(examples[j][i],2)
            dic[j].append(arr[0])
            dic[j].append(arr[1])
    return dic

In [147]:
test = tokenized_datasets.map(split,remove_columns=["input_ids","token_type_ids","attention_mask","labels","texts"], batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [148]:
full_train_dataset = test["train"]
full_eval_dataset = test["test"]

In [149]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [150]:
from datasets import load_metric
import numpy as np
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

In [151]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [152]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=100,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [153]:
trainer = Trainer(model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset,compute_metrics=compute_metrics, data_collator=data_collator,
    tokenizer=tokenizer)

In [154]:
trainer.train()

***** Running training *****
  Num examples = 48
  Num Epochs = 100
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 800


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Location F1,Location City F1,Location Country F1,Misc Comments F1,Misc Friends F1,Misc Likes F1,Misc Post F1,Organization F1,Person Firstname F1,Person Lastname F1
1,No log,0.606074,0.360000,0.493151,0.416185,0.853415,0.000000,0.909091,0.666667,0.117647,0.426230,0.296296,0.000000,0.000000,0.666667,0.705882
2,No log,0.728021,0.397260,0.397260,0.397260,0.850083,0.000000,0.727273,0.000000,0.153846,0.530612,0.333333,0.000000,0.000000,0.533333,0.615385
3,No log,0.536627,0.472222,0.465753,0.468966,0.916713,0.000000,0.769231,0.666667,0.000000,0.509091,0.210526,0.000000,0.000000,0.705882,0.750000
4,No log,0.664713,0.342105,0.534247,0.417112,0.845086,0.000000,0.625000,0.666667,0.200000,0.470588,0.333333,0.000000,0.000000,0.705882,0.666667
5,No log,0.655408,0.440476,0.506849,0.471338,0.859522,0.000000,0.833333,0.666667,0.222222,0.555556,0.285714,0.000000,0.000000,0.705882,0.750000
6,No log,1.061471,0.375000,0.534247,0.440678,0.781233,0.000000,0.833333,0.666667,0.363636,0.588235,0.181818,0.076923,0.153846,0.705882,0.750000
7,No log,0.708609,0.402174,0.506849,0.448485,0.865075,0.000000,0.615385,0.666667,0.111111,0.555556,0.370370,0.000000,0.000000,0.625000,0.857143
8,No log,0.830030,0.366972,0.547945,0.439560,0.820100,0.000000,0.769231,0.666667,0.222222,0.561404,0.400000,0.000000,0.000000,0.705882,0.750000
9,No log,0.707161,0.541667,0.534247,0.537931,0.914492,0.000000,0.833333,0.666667,0.285714,0.627451,0.320000,0.000000,0.000000,0.705882,0.750000
10,No log,0.992233,0.396040,0.547945,0.459770,0.820655,0.000000,0.909091,0.666667,0.222222,0.533333,0.380952,0.090909,0.000000,0.705882,0.750000


***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 12
  Batch size = 6
***** Running Evaluation *****
  Num examples = 

TrainOutput(global_step=800, training_loss=0.006312715150415898, metrics={'train_runtime': 235.3289, 'train_samples_per_second': 20.397, 'train_steps_per_second': 3.399, 'total_flos': 2488301812482408.0, 'train_loss': 0.006312715150415898, 'epoch': 100.0})

In [139]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: texts.
***** Running Evaluation *****
  Num examples = 6
  Batch size = 6


{'eval_loss': 1.4229483604431152,
 'eval_overall_precision': 0.2916666666666667,
 'eval_overall_recall': 0.49295774647887325,
 'eval_overall_f1': 0.36649214659685864,
 'eval_overall_accuracy': 0.8171530673019655,
 'eval_LOCATION_f1': 0.0,
 'eval_LOCATION_CITY_f1': 0.625,
 'eval_LOCATION_COUNTRY_f1': 0.6666666666666666,
 'eval_MISC_COMMENTS_f1': 0.11764705882352941,
 'eval_MISC_FRIENDS_f1': 0.56,
 'eval_MISC_LIKES_f1': 0.16,
 'eval_MISC_POST_f1': 0.0,
 'eval_ORGANIZATION_f1': 0.0,
 'eval_PERSON_FIRSTNAME_f1': 0.7058823529411764,
 'eval_PERSON_LASTNAME_f1': 0.75,
 'eval_runtime': 0.1658,
 'eval_samples_per_second': 36.188,
 'eval_steps_per_second': 6.031,
 'epoch': 100.0}

In [31]:
model.save_pretrained('./NEWS_NER_2/')

Configuration saved in ./NEWS_NER_2/config.json
Model weights saved in ./NEWS_NER_2/pytorch_model.bin


In [32]:
tokenizer.save_pretrained('./NEWS_TOK_2/')

tokenizer config file saved in ./NEWS_TOK_2/tokenizer_config.json
Special tokens file saved in ./NEWS_TOK_2/special_tokens_map.json


('./NEWS_TOK_2/tokenizer_config.json',
 './NEWS_TOK_2/special_tokens_map.json',
 './NEWS_TOK_2/vocab.txt',
 './NEWS_TOK_2/added_tokens.json',
 './NEWS_TOK_2/tokenizer.json')

In [59]:
{'eval_loss': 0.6682612299919128,
 'eval_overall_precision': 0.38461538461538464,
 'eval_overall_recall': 0.6395348837209303,
 'eval_overall_f1': 0.480349344978166,
 'eval_overall_accuracy': 0.8915917503966155,
 'eval_LOCATION_f1': 0.0,
 'eval_LOCATION_ASSOSIATION_f1': 0.0,
 'eval_LOCATION_CITY_f1': 0.631578947368421,
 'eval_LOCATION_COUNTRY_f1': 0.0,
 'eval_MISC_COMMENTS_f1': 0.14814814814814814,
 'eval_MISC_FRIENDS_f1': 0.627906976744186,
 'eval_MISC_LIKES_f1': 0.5517241379310345,
 'eval_MISC_POST_f1': 0.0 ,
 'eval_ORGANIZATION_f1': 0.0,
 'eval_PERSON_FIRSTNAME_f1': 0.923076923076923,
 'eval_PERSON_LASTNAME_f1': 1.0,
 'eval_PERSON_NAME_f1': 0.0,
 'eval_runtime': 0.1664,
 'eval_samples_per_second': 36.065,
 'eval_steps_per_second': 6.011,
 'epoch': 100.0}

{'eval_loss': 0.11371712386608124,
 'eval_overall_precision': 0.8902077151335311,
 'eval_overall_recall': 0.9433962264150944,
 'eval_overall_f1': 0.916030534351145,
 'eval_overall_accuracy': 0.982909090909091,
 'eval_LOCATION_f1': 0.2727272727272727,
 'eval_LOCATION_ASSOSIATION_f1': 0.6666666666666666,
 'eval_LOCATION_CITY_f1': 0.8823529411764706,
 'eval_LOCATION_COUNTRY_f1': 0.5,
 'eval_LOCATION_STREET_f1': 1.0,
 'eval_ORGANIZATION_f1': 0.6086956521739131,
 'eval_PERSON_AGE_f1': 1.0,
 'eval_PERSON_FIRSTNAME_f1': 1.0,
 'eval_PERSON_LASTNAME_f1': 0.9600000000000001,
 'eval_SENTENCE_END_f1': 0.9822064056939502,
 'eval_runtime': 0.2295,
 'eval_samples_per_second': 43.569,
 'eval_steps_per_second': 8.714,
 'epoch': 100.0}